<a href="https://colab.research.google.com/github/Philewj/CS-695-CNN-Implementation-MNIST/blob/main/CS_695_Project2_CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Implementing CNN for the MNIST Dataset

## Building Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

### Load Data

In [ ]:
#Covert MNIST Image Files into Tensors of 4 Dimensions (# of images, Height, Width, Color Channels)
transform = transforms.ToTensor()
#Load Training Data
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)
#Load Testing Data
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [ ]:
#Verifying Training Data
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
#Verifying Testing Data
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

### Build Model

In [24]:
#Model Class
class ConvolutionalNeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    # Create 3 Convolutional Layers
    self.convolution_layer1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3,stride=1)
    self.convolution_layer2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3,stride=1)

    #Fully Connected Neural Layer3
    self.fc1 = nn.Linear(in_features=5*5*16, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=85) # decrease the number of out features (can be an arbitrary number)
    self.fc3 = nn.Linear(in_features=85, out_features=55) # is the number of classes in the dataset
    self.fc4 = nn.Linear(in_features=55, out_features=10) # is the number of classes in the dataset

  def forward(self, X):
    # Pass through Convolutional Layer 1
    X = F.relu(self.convolution_layer1(X))
    X = F.max_pool2d(X, 2, 2)
    # Pass through Convolutional Layer 2
    X = F.relu(self.convolution_layer2(X))
    X = F.max_pool2d(X, 2, 2)


    #Re-View data to flatten it out
    X = X.view(-1, 16*5*5) # -1 so we can vary batch size

    #Fully Connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = F.relu(self.fc3(X))
    X = self.fc4(X)
    return F.log_softmax(X, dim=1)

In [ ]:
#Create an Intance of the Model
#Create a manual seed
torch.manual_seed(41)
model = ConvolutionalNeuralNetwork()
model

ConvolutionalNeuralNetwork(
  (convolution_layer1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (convolution_layer2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=85, bias=True)
  (fc3): Linear(in_features=85, out_features=55, bias=True)
  (fc4): Linear(in_features=55, out_features=10, bias=True)
)

In [ ]:
#Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Training Model

In [ ]:
#Keep track of how long it takes to train the model
import time
start_time = time.time()

#Create a small batch size for images ...10
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

#Create Variables to track things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []


#Create For Loops of Epochs
for i in range(epochs) :
  trn_corr = 0
  tst_corr = 0

  #Train
  for b, (X_train, y_train) in enumerate(train_loader):
    b+=1 #start our batches at 1
    y_pred = model(X_train)   # Get predicted values from training set. Not Flattened.2D
    loss = criterion(y_pred, y_train) # Calculate Loss, Compare the predictions to correct answers in y_train

    predicted = torch.max(y_pred.data,1)[1] #add up the number of correct predictions
    batch_corr = (predicted == y_train).sum() # How many we got correct from this batch. True =1, False = 0, sum those up
    trn_corr += batch_corr #Keep track as we go along in training


    #Update our parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



    #Print results
    if b%600 ==0 :
      print(f'Epoch: {i}  Batch: {b}  Loss: {loss.item()}')
  train_losses.append(loss)
  train_correct.append(trn_corr)

  #Test
  with torch.no_grad():  # No Gradient so that we don't update our weights and biases with test data
    for b, (X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data, 1)[1] #Adding up correct predictions
      tst_corr += (predicted == y_test).sum() #True =1 False = 0 and sum them


    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)




current_time = time.time()
total_time = current_time - start_time
print(f'Training Took: {total_time/60}  minutes!')

Epoch: 0  Batch: 600  Loss: 0.6502459049224854
Epoch: 0  Batch: 1200  Loss: 0.19238707423210144
Epoch: 0  Batch: 1800  Loss: 0.199004128575325
Epoch: 0  Batch: 2400  Loss: 0.1920306235551834
Epoch: 0  Batch: 3000  Loss: 0.037895798683166504
Epoch: 0  Batch: 3600  Loss: 0.0013977289199829102
Epoch: 0  Batch: 4200  Loss: 0.19188503921031952
Epoch: 0  Batch: 4800  Loss: 0.014569433405995369
Epoch: 0  Batch: 5400  Loss: 0.034884121268987656
Epoch: 0  Batch: 6000  Loss: 0.10975992679595947
Epoch: 1  Batch: 600  Loss: 0.0022547177504748106
Epoch: 1  Batch: 1200  Loss: 0.032249368727207184
Epoch: 1  Batch: 1800  Loss: 0.028194641694426537
Epoch: 1  Batch: 2400  Loss: 0.015163414180278778
Epoch: 1  Batch: 3000  Loss: 0.15465423464775085
Epoch: 1  Batch: 3600  Loss: 0.0036955750547349453
Epoch: 1  Batch: 4200  Loss: 0.010395861230790615
Epoch: 1  Batch: 4800  Loss: 0.10470445454120636
Epoch: 1  Batch: 5400  Loss: 0.09734562784433365
Epoch: 1  Batch: 6000  Loss: 0.09871844947338104
Epoch: 2  Bat

####Training 2